In [1]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch("http://localhost:9200")

In [3]:
# 删除索引

es.indices.delete(index="vector_index", ignore=404) 

/var/folders/f2/s10901_5219dbnccmr_04k3m0000gn/T/ipykernel_64863/1868941633.py:3: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index="vector_index", ignore=404)


ObjectApiResponse({'acknowledged': True})

In [4]:
# 1 定义索引结构

body = {
    "mappings":{
        "properties":{
            "name":{
                "type":"text"
            },
            "embedding":{
                "type":"dense_vector",
                "dims": 3
            }
        }
    }
}

In [5]:
# 2 创建索引

es.indices.create(index="vector_index",body=body)

/var/folders/f2/s10901_5219dbnccmr_04k3m0000gn/T/ipykernel_64863/4173687513.py:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index="vector_index",body=body)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'vector_index'})

In [6]:
# 3 批量插入数据
from elasticsearch import helpers

actions = [
    {"_index":"vector_index",
     "_source":{"name":"苹果",
                "embedding":[0.2,0.1,0.4]}},
    {"_index":"vector_index",
     "_source":{"name":"小船",
                "embedding":[0.7,0.2,0.6]}},
    {"_index":"vector_index",
     "_source":{"name":"香蕉",
                "embedding":[0.3,0.1,0.3]}}
]

helpers.bulk(es, actions)

(3, [])

In [7]:
# 4 查询所有记录

result = es.search(index="vector_index", body={"query":{"match_all":{}}})

result

/var/folders/f2/s10901_5219dbnccmr_04k3m0000gn/T/ipykernel_64863/1862223120.py:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="vector_index", body={"query":{"match_all":{}}})


ObjectApiResponse({'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'vector_index', '_id': 'W1L6GIAB8jcpMxT8Cr4k', '_score': 1.0, '_source': {'name': '苹果', 'embedding': [0.2, 0.1, 0.4]}}, {'_index': 'vector_index', '_id': 'XFL6GIAB8jcpMxT8Cr4k', '_score': 1.0, '_source': {'name': '小船', 'embedding': [0.7, 0.2, 0.6]}}, {'_index': 'vector_index', '_id': 'XVL6GIAB8jcpMxT8Cr4k', '_score': 1.0, '_source': {'name': '香蕉', 'embedding': [0.3, 0.1, 0.3]}}]}})

In [8]:
# 5 根据向量计算相似性

temp = "橘子"
temp_emb = [0.2,0.2,0.4]

query = {
    "query":{
        "script_score":{
            "query":{
                "match_all":{}
            },
            "script":{
                "source":"cosineSimilarity(params.queryVector, 'embedding') + 1.0",
                "params":{
                    "queryVector":temp_emb
                }
            }
        }
    }
}

response = es.search(index="vector_index", body=query)

/var/folders/f2/s10901_5219dbnccmr_04k3m0000gn/T/ipykernel_64863/1723803535.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index="vector_index", body=query)


In [9]:
# 6 输出结果

response

ObjectApiResponse({'took': 7, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 1.9799578, 'hits': [{'_index': 'vector_index', '_id': 'W1L6GIAB8jcpMxT8Cr4k', '_score': 1.9799578, '_source': {'name': '苹果', 'embedding': [0.2, 0.1, 0.4]}}, {'_index': 'vector_index', '_id': 'XVL6GIAB8jcpMxT8Cr4k', '_score': 1.9365858, '_source': {'name': '香蕉', 'embedding': [0.3, 0.1, 0.3]}}, {'_index': 'vector_index', '_id': 'XFL6GIAB8jcpMxT8Cr4k', '_score': 1.9087589, '_source': {'name': '小船', 'embedding': [0.7, 0.2, 0.6]}}]}})

In [10]:
response['hits']['hits']

[{'_index': 'vector_index',
  '_id': 'W1L6GIAB8jcpMxT8Cr4k',
  '_score': 1.9799578,
  '_source': {'name': '苹果', 'embedding': [0.2, 0.1, 0.4]}},
 {'_index': 'vector_index',
  '_id': 'XVL6GIAB8jcpMxT8Cr4k',
  '_score': 1.9365858,
  '_source': {'name': '香蕉', 'embedding': [0.3, 0.1, 0.3]}},
 {'_index': 'vector_index',
  '_id': 'XFL6GIAB8jcpMxT8Cr4k',
  '_score': 1.9087589,
  '_source': {'name': '小船', 'embedding': [0.7, 0.2, 0.6]}}]